# model

> DiffPASS models for optimizing MSA pairing

In [ ]:
#| default_exp model

In [ ]:
#| hide

from nbdev.showdoc import *

In [ ]:
#| export

# Stdlib imports
from collections.abc import Iterable, Sequence
from typing import Optional, Union, Iterator, Literal
from copy import deepcopy
from warnings import warn

# NumPy
import numpy as np

# PyTorch
import torch
from torch.nn import Module, ParameterList, Parameter, CosineSimilarity
from torch.nn.functional import softmax

# DiffPASS imports
from diffpass.base import EnsembleMixin
from diffpass.gumbel_sinkhorn_ops import gumbel_sinkhorn, gumbel_matching
from diffpass.entropy_ops import (
    smooth_mean_one_body_entropy,
    smooth_mean_two_body_entropy,
)
from diffpass.constants import get_blosum62_data
from diffpass.sequence_similarity_ops import (
    smooth_hamming_similarities_dot,
    smooth_hamming_similarities_cdist,
    smooth_substitution_matrix_similarities,
    soft_reciprocal_best_hits,
    hard_reciprocal_best_hits,
)

In [ ]:
#| export

def _consecutive_slices_from_sizes(group_sizes: Optional[Sequence[int]]) -> list[slice]:
    if group_sizes is None:
        return [slice(None)]
    cumsum = np.cumsum(group_sizes).tolist()

    return [slice(start, end) for start, end in zip([0] + cumsum, cumsum)]

## Soft permutations (Gumbel-Sinkhorn operator)

In [ ]:
#| export

class GeneralizedPermutation(Module, EnsembleMixin):
    """Generalized permutation layer implementing both soft and hard permutations."""

    def __init__(
        self,
        *,
        group_sizes: Iterable[int],
        ensemble_shape: Optional[Iterable[int]] = None,
        fixed_matchings: Optional[Sequence[Sequence[Sequence[int]]]] = None,
        tau: Union[float, torch.Tensor] = 1.0,
        tau_dim_in_ensemble: Optional[int] = None,
        n_iter: int = 10,
        noise: bool = False,
        noise_factor: float = 1.0,
        noise_std: bool = False,
        mode: Literal["soft", "hard"] = "soft",
    ) -> None:
        super().__init__()
        self.group_sizes = tuple(s for s in group_sizes)
        self.ensemble_shape = torch.Size(
            ensemble_shape if ensemble_shape is not None else []
        )
        self._validate_fixed_matchings(fixed_matchings)
        self.fixed_matchings = fixed_matchings
        self.n_iter = n_iter
        self.noise = noise
        self.noise_factor = noise_factor
        self.noise_std = noise_std

        # By default, initialize all parametrization matrices to zero
        # If `ensemble_shape` is not None, use the first len(ensemble_shape) dimensions
        # as "batch" dimensions.
        self.nonfixed_group_sizes_ = (
            tuple(
                s - num_efm
                for s, num_efm in zip(
                    self.group_sizes, self._effective_number_fixed_matchings
                )
            )
            if self.fixed_matchings
            else self.group_sizes
        )
        self.log_alphas = ParameterList(
            [
                Parameter(torch.zeros(*self.ensemble_shape, s, s, dtype=torch.float32))
                for s in self.nonfixed_group_sizes_
            ]
        )

        tau = self._validate_ensemble_param(
            param=tau,
            dim_in_ensemble=tau_dim_in_ensemble,
            param_name="tau",
            ensemble_shape=self.ensemble_shape,
            n_dims_per_instance=2,
        )
        self.register_buffer("tau", tau)
        self.tau_dim_in_ensemble = tau_dim_in_ensemble
        self.mode = mode

    def _validate_fixed_matchings(
        self, fixed_matchings: Optional[Sequence[Sequence[Sequence[int]]]] = None
    ) -> None:
        if fixed_matchings:
            if len(fixed_matchings) != len(self.group_sizes):
                raise ValueError(
                    "If `fixed_matchings` is provided, it must have the same length as "
                    "`group_sizes`."
                )
            for s, fm in zip(self.group_sizes, fixed_matchings):
                if not fm:
                    continue
                if any([len(p) != 2 for p in fm]):
                    raise ValueError(
                        "All fixed matchings must be pairs of indices (i, j)."
                    )
                if any(min(i, j) < 0 or max(i, j) >= s for i, j in fm):
                    raise ValueError(
                        "All fixed matchings must be within the range of the corresponding "
                        "group size."
                    )
            self._effective_number_fixed_matchings = []
            self._effective_fixed_matchings_zip = []
            for idx, (s, fm) in enumerate(zip(self.group_sizes, fixed_matchings)):
                if fm:
                    num_fm = len(fm)
                    fm_zip = list(zip(*fm))
                else:
                    num_fm = 0
                    fm_zip = ((), ())
                complement = s - num_fm  # Effectively fully fixed when complement <= 1
                is_fully_fixed = complement <= 1
                num_efm = s - (s - num_fm) * (not is_fully_fixed)
                self._effective_number_fixed_matchings.append(num_efm)
                if is_fully_fixed:
                    mask = torch.zeros(*self.ensemble_shape, s, s, dtype=torch.bool)
                    if complement:
                        possible_idxs = set(range(s))
                        fm_zip[0] += tuple((possible_idxs - set(fm_zip[0])))
                        fm_zip[1] += tuple((possible_idxs - set(fm_zip[1])))
                else:
                    mask = torch.ones(*self.ensemble_shape, s, s, dtype=torch.bool)
                    for i, j in fm:
                        mask[..., j, :] = False
                        mask[..., :, i] = False
                self.register_buffer(f"_not_fixed_masks_{idx}", mask)
                self._effective_fixed_matchings_zip.append(fm_zip)

    @property
    def _not_fixed_masks(self) -> list[torch.Tensor]:
        return [
            getattr(self, f"_not_fixed_masks_{idx}")
            for idx in range(len(self.group_sizes))
        ]

    @property
    def mode(self) -> str:
        return self._mode

    @mode.setter
    def mode(self, value) -> None:
        value = value.lower()
        if value not in ["soft", "hard"]:
            raise ValueError("mode must be either 'soft' or 'hard'.")
        self._mode = value.lower()
        _mats_fn_no_fixed = getattr(self, f"_{self._mode}_mats")
        self._mats_fn = (
            _mats_fn_no_fixed
            if not self.fixed_matchings
            else self._impl_fixed_matchings(_mats_fn_no_fixed)
        )

    def soft_(self) -> None:
        self.mode = "soft"

    def hard_(self) -> None:
        self.mode = "hard"

    def _impl_fixed_matchings(self, func: callable) -> callable:
        """Include fixed matchings in the Gumbel-Sinkhorn or Gumbel-matching operators."""

        def wrapper(gen: Iterator[torch.Tensor]) -> Iterator[torch.Tensor]:
            for s, mat, (row_group, col_group), mask in zip(
                self.group_sizes,
                gen,
                self._effective_fixed_matchings_zip,
                self._not_fixed_masks,
            ):
                mat_all = torch.zeros(
                    *self.ensemble_shape,
                    s,
                    s,
                    dtype=mat.dtype,
                    layout=mat.layout,
                    device=mat.device,
                )
                # mat_all[j, i] = 1 means that row i becomes row j under a permutation,
                # using our conventions
                mat_all[..., col_group, row_group] = 1
                mat_all.masked_scatter_(mask.to(torch.bool), mat)
                yield mat_all

        return lambda: wrapper(func())

    def _soft_mats(self) -> Iterator[torch.Tensor]:
        """Evaluate the Gumbel-Sinkhorn operator on the current `log_alpha` parameters."""
        return (
            gumbel_sinkhorn(
                log_alpha,
                tau=self.tau,
                n_iter=self.n_iter,
                noise=self.noise,
                noise_factor=self.noise_factor,
                noise_std=self.noise_std,
            )
            for log_alpha in self.log_alphas
        )

    def _hard_mats(self) -> Iterator[torch.Tensor]:
        """Evaluate the Gumbel-matching operator on the current `log_alpha` parameters."""
        return (
            gumbel_matching(
                log_alpha,
                noise=self.noise,
                noise_factor=self.noise_factor,
                noise_std=self.noise_std,
                unbias_lsa=True,
            )
            for log_alpha in self.log_alphas
        )

    def forward(self) -> list[torch.Tensor]:
        """Compute the soft/hard permutations according to ``self._mats_fn.``"""
        mats = self._mats_fn()

        return list(mats)


class EnsembleMatrixApply(Module):
    """Apply batches of matrices to chunks of a tensor of shape (n_samples, length, alphabet_size)
    and collate the results."""

    def __init__(self, group_sizes: Iterable[int]) -> None:
        super().__init__()
        self.group_sizes = tuple(s for s in group_sizes)
        self._group_slices = _consecutive_slices_from_sizes(self.group_sizes)

    def forward(self, x: torch.Tensor, *, mats: Sequence[torch.Tensor]) -> torch.Tensor:
        ensemble_shape = mats[0].shape[:-2]
        out = torch.full_like(x, torch.nan).repeat(*ensemble_shape, 1, 1, 1)
        for mats_this_group, sl in zip(mats, self._group_slices):
            out[..., sl, :, :].copy_(
                torch.tensordot(mats_this_group, x[sl, :, :], dims=1)
            )

        return out

In [ ]:
show_doc(GeneralizedPermutation)

In [ ]:
# Test for GeneralizedPermutation

def test_generalizedpermutation_shape(*, length, alphabet_size, init_kwargs):
    group_sizes = init_kwargs["group_sizes"]
    n_samples = sum(group_sizes)

    x = torch.randn(n_samples, length, alphabet_size)
    perm = GeneralizedPermutation(**init_kwargs)
    mats = perm()
    ens_apply = EnsembleMatrixApply(group_sizes)
    y = ens_apply(x, mats=mats)

    assert y.shape == perm.ensemble_shape + x.shape
    assert y.requires_grad

    perm.hard_()
    assert perm.mode == "hard"


test_generalizedpermutation_shape(
    length=5,
    alphabet_size=10,
    init_kwargs={
        "group_sizes": [3, 2, 4],
        "ensemble_shape": (2, 3, 3, 3),
        "fixed_matchings": [[[0, 1]], [[0, 0]], [[1, 0], [2, 3]]],
        "tau": torch.tensor([1., 0.1, 3.]),
        "tau_dim_in_ensemble": 2
    }
)

## Two-body entropy loss

In [ ]:
#| export

class TwoBodyEntropyLoss(Module):
    def __init__(self):
        super().__init__()

    def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        return smooth_mean_two_body_entropy(x, y)


class MILoss(Module):
    def __init__(self):
        super().__init__()

    def forward(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        return smooth_mean_two_body_entropy(x, y) - smooth_mean_one_body_entropy(x)

In [ ]:
# Test for TwoBodyEntropyScore

def test_twobodyentropyloss(
        *,
        n_samples, length_x, length_y, alphabet_size, ensemble_shape
):
    x = torch.randn(
        *ensemble_shape, n_samples, length_x, alphabet_size,
        requires_grad=True
    )
    y = torch.randn(n_samples, length_y, alphabet_size)
    x_soft = softmax(x, dim=-1)
    y_soft = softmax(y, dim=-1)
    two_body_entropy_loss = TwoBodyEntropyLoss()
    loss = two_body_entropy_loss(x_soft, y_soft)

    assert loss.shape == ensemble_shape
    assert loss.requires_grad

    # In the following scenario, the score should be close to log2(alphabet_size)
    x_almost_hard = softmax(x / 1e-5, dim=-1)
    first_x_almost_hard_length_1 = x_almost_hard[(0,) * len(ensemble_shape)][:, :1, :]
    loss = two_body_entropy_loss(
        first_x_almost_hard_length_1, first_x_almost_hard_length_1
    )

    torch.testing.assert_close(
        loss, torch.log2(torch.tensor(alphabet_size)), atol=1e-3, rtol=1e-7
    )


test_twobodyentropyloss(
    n_samples=10_000,
    length_x=3,
    length_y=4,
    alphabet_size=3,
    ensemble_shape=(2, 3),
)

## Sequence similarities (Hamming and Blosum62)

In [ ]:
#| export

class HammingSimilarities(Module):
    def __init__(
        self,
        *,
        group_sizes: Optional[Iterable[int]] = None,
        use_dot: bool = True,
        p: Optional[float] = None,
    ) -> None:
        super().__init__()
        self.group_sizes = (
            tuple(s for s in group_sizes) if group_sizes is not None else None
        )
        self.use_dot = use_dot
        self.p = p

        if self.use_dot:
            if self.p is not None:
                warn("Since a `p` was provided, `use_dot` will be ignored.")
            self._similarities_fn = smooth_hamming_similarities_dot
            self._similarities_fn_kwargs = {}
        else:
            if self.p is None:
                raise ValueError("If `use_dot` is False, `p` must be provided.")
            self._similarities_fn = smooth_hamming_similarities_cdist
            self._similarities_fn_kwargs = {"p": p}

        self._group_slices = _consecutive_slices_from_sizes(self.group_sizes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        size = x.shape[:-3] + (x.shape[-3],) * 2
        out = torch.full(
            size, torch.nan, dtype=x.dtype, layout=x.layout, device=x.device
        )
        for sl in self._group_slices:
            out[..., sl, sl].copy_(
                self._similarities_fn(x[..., sl, :, :], **self._similarities_fn_kwargs)
            )

        return out


class Blosum62Similarities(Module):
    def __init__(
        self,
        *,
        group_sizes: Optional[Iterable[int]] = None,
        use_scoredist: bool = False,
        aa_to_int: Optional[dict[str, int]] = None,
        gaps_as_stars: bool = True,
    ) -> None:
        super().__init__()
        self.group_sizes = (
            tuple(s for s in group_sizes) if group_sizes is not None else None
        )
        self.use_scoredist = use_scoredist
        self.aa_to_int = aa_to_int
        self.gaps_as_stars = gaps_as_stars

        blosum62_data = get_blosum62_data(
            aa_to_int=self.aa_to_int, gaps_as_stars=self.gaps_as_stars
        )
        self.register_buffer("subs_mat", blosum62_data.mat)
        self.expected_value = blosum62_data.expected_value

        self._group_slices = _consecutive_slices_from_sizes(self.group_sizes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        size = x.shape[:-3] + (x.shape[-3],) * 2
        out = torch.full(
            size, torch.nan, dtype=x.dtype, layout=x.layout, device=x.device
        )
        for sl in self._group_slices:
            out[..., sl, sl].copy_(
                smooth_substitution_matrix_similarities(
                    x[..., sl, :, :],
                    subs_mat=self.subs_mat,
                    expected_value=self.expected_value,
                    use_scoredist=self.use_scoredist,
                )
            )

        return out

In [ ]:
# Tests for HammingSimilarities and Blosum62Similarities

def test_similarities(
        *,
        cls,
        ensemble_shape, length, alphabet_size,
        init_kwargs
):
    group_sizes = init_kwargs["group_sizes"]
    n_samples = sum(group_sizes)

    x = torch.randn(
        *ensemble_shape, n_samples, length, alphabet_size,
        requires_grad=True
    )
    x_soft = softmax(x, dim=-1)

    _init_kwargs = deepcopy(init_kwargs)
    _init_kwargs["group_sizes"] = None
    similarities = cls(**_init_kwargs)
    out_all = similarities(x_soft)

    assert out_all.shape == ensemble_shape + (n_samples, n_samples)

    similarities = cls(**init_kwargs)
    out = similarities(x_soft)

    for sl in similarities._group_slices:
        assert torch.allclose(
            out[..., sl, sl], out_all[..., sl, sl]
        )


test_similarities(
    cls=HammingSimilarities,
    ensemble_shape=(3, 2),
    length=3,
    alphabet_size=10,
    init_kwargs={"group_sizes": [3, 2, 4], "use_dot": False, "p": 1.}
)

test_similarities(
    cls=Blosum62Similarities,
    ensemble_shape=(3, 2),
    length=3,
    alphabet_size=21,
    init_kwargs={"group_sizes": [3, 2, 4]}
)

## Reciprocal best hits

In [ ]:
#| export

class ReciprocalBestHits(Module, EnsembleMixin):
    def __init__(
        self,
        *,
        group_sizes: Iterable[int],
        ensemble_shape: Optional[Iterable[int]] = None,
        tau: Union[float, torch.Tensor] = 0.1,
        tau_dim_in_ensemble: Optional[int] = None,
        mode: Literal["soft", "hard"] = "soft",
    ) -> None:
        super().__init__()
        self.group_sizes = tuple(s for s in group_sizes)
        self.ensemble_shape = torch.Size(
            ensemble_shape if ensemble_shape is not None else []
        )

        self._group_slices = _consecutive_slices_from_sizes(self.group_sizes)

        tau = self._validate_ensemble_param(
            param=tau,
            param_name="tau",
            dim_in_ensemble=tau_dim_in_ensemble,
            ensemble_shape=self.ensemble_shape,
            n_dims_per_instance=2,
        )
        self.register_buffer("tau", tau)
        self.tau_dim_in_ensemble = tau_dim_in_ensemble

        self.mode = mode

    @property
    def mode(self) -> str:
        return self._mode

    @mode.setter
    def mode(self, value) -> None:
        value = value.lower()
        if value not in ["soft", "hard"]:
            raise ValueError("`mode` must be either 'soft' or 'hard'.")
        self._mode = value.lower()
        self._rbh_fn = getattr(self, f"_{self._mode}_rbh_fn")

    def soft_(self) -> None:
        self.mode = "soft"

    def hard_(self) -> None:
        self.mode = "hard"

    def _soft_rbh_fn(self, similarities: torch.Tensor) -> torch.Tensor:
        """Compute soft reciprocal best hits."""
        return soft_reciprocal_best_hits(
            similarities,
            group_slices=self._group_slices,
            tau=self.tau,
        )

    def _hard_rbh_fn(self, similarities: torch.Tensor) -> torch.Tensor:
        """Compute hard reciprocal best hits."""
        return hard_reciprocal_best_hits(
            similarities,
            group_slices=self._group_slices,
        )

    def prepare_fixed(self, similarities: torch.Tensor) -> torch.Tensor:
        """Repeat a similarity matrix along the tau ensemble dimension and unsqueeze."""
        assert similarities.ndim == 2
        if self.mode == "soft" and self.tau.ndim:
            if self.tau_dim_in_ensemble is None:
                raise ValueError(
                    "If using soft reciprocal best hits and a 1D `tau`, "
                    "`tau_dim_in_ensemble` must be provided."
                )
            n_ensemble_dims = len(self.ensemble_shape)
            # FIXME (self.tau.shape[self.tau_dim_in_ensemble],) is a hack, it's there
            #  because we lost the original 1D tau
            new_shape = (
                (1,) * self.tau_dim_in_ensemble
                + (self.tau.shape[self.tau_dim_in_ensemble],)
                + (1,) * (n_ensemble_dims - self.tau_dim_in_ensemble - 1)
                + (1,) * 2
            )
            similarities = similarities.repeat(*new_shape)

        return similarities

    def forward(self, similarities: torch.Tensor) -> torch.Tensor:
        # Input validation
        if self.mode == "soft" and self.tau.ndim:
            if similarities.ndim != len(self.ensemble_shape) + 2:
                raise ValueError(
                    f"If using soft reciprocal best hits and a 1D `tau`, the input must have the "
                    f"same number of dimensions as the ensemble shape plus 2 = "
                    f"{len(self.ensemble_shape) + 2}, not {similarities.ndim}."
                )
        else:
            if similarities.ndim != 2:
                raise ValueError(
                    f"If using hard reciprocal best hits or a 0D `tau`, "
                    f"the input must have 2 dimensions, not {similarities.ndim}."
                )

        return self._rbh_fn(similarities)

In [ ]:
#| export

class InterGroupLoss(Module, EnsembleMixin):
    def __init__(
        self,
        *,
        group_sizes: Iterable[int],
        score_fn: Union[callable, None] = None,
    ) -> None:
        super().__init__()
        self.group_sizes = tuple(s for s in group_sizes)
        self.score_fn = CosineSimilarity(dim=-1) if score_fn is None else score_fn

        self._group_slices = _consecutive_slices_from_sizes(self.group_sizes)
        diag_blocks_mask = torch.block_diag(
            *[torch.ones((s, s), dtype=torch.bool) for s in self.group_sizes]
        )
        self._upper_no_diag_blocks_mask = torch.triu(~diag_blocks_mask)

    def forward(
        self, similarities_x: torch.Tensor, similarities_y: torch.Tensor
    ) -> torch.Tensor:
        # Input validation
        assert similarities_x.ndim >= 2 and similarities_y.ndim >= 2

        scores = self.score_fn(
            similarities_x[..., self._upper_no_diag_blocks_mask],
            similarities_y[..., self._upper_no_diag_blocks_mask],
        )
        loss = -scores

        return loss


class IntraGroupLoss(Module):
    def __init__(
        self,
        *,
        group_sizes: Optional[Iterable[int]] = None,
        score_fn: Union[callable, None] = None,
    ) -> None:
        super().__init__()
        self.group_sizes = (
            tuple(s for s in group_sizes) if group_sizes is not None else None
        )
        self.score_fn = CosineSimilarity(dim=-1) if score_fn is None else score_fn

        if self.group_sizes is not None:
            diag_blocks_mask = torch.block_diag(
                *[torch.ones((s, s), dtype=torch.bool) for s in self.group_sizes]
            )
            # Exclude main diagonal
            self._upper_diag_blocks_mask = torch.triu(diag_blocks_mask, diagonal=1)
        else:
            self._upper_diag_blocks_mask = None

    def forward(
        self, similarities_x: torch.Tensor, similarities_y: torch.Tensor
    ) -> torch.Tensor:
        assert similarities_x.ndim >= 2 and similarities_y.ndim >= 2

        if self._upper_diag_blocks_mask is None:
            mask = torch.triu(
                torch.ones(
                    similarities_x.shape[-2:],
                    dtype=torch.bool,
                    layout=similarities_x.layout,
                    device=similarities_x.device,
                ),
                diagonal=1,
            )
        else:
            mask = self._upper_diag_blocks_mask

        scores = self.score_fn(
            similarities_x[..., mask],
            similarities_y[..., mask],
        )
        loss = -scores

        return loss

In [ ]:
# Test for ReciprocalBestHits, InterGroupLoss and IntraGroupLoss

def test_sequence_similarity_losses(
        *,
        group_sizes,
        length_x, length_y, alphabet_size,
        extra_init_kwargs_rbh, extra_init_kwargs_loss
):
    similarities = HammingSimilarities(group_sizes=None)
    reciprocal_best_hits = ReciprocalBestHits(group_sizes=group_sizes, **extra_init_kwargs_rbh)
    n_samples = sum(group_sizes)

    y = torch.randn(n_samples, length_y, alphabet_size)
    y.scatter_(-1, y.argmax(dim=-1, keepdim=True), 1.)
    similarities_y = similarities(y)
    reciprocal_best_hits.hard_()
    reciprocal_best_hits_y = reciprocal_best_hits(similarities_y)
    reciprocal_best_hits.soft_()

    ensemble_shape = extra_init_kwargs_rbh["ensemble_shape"]
    x = torch.randn(
        *ensemble_shape, n_samples, length_x, alphabet_size,
        requires_grad=True
    )
    x_soft = softmax(x, dim=-1)
    similarities_x = similarities(x_soft)
    reciprocal_best_hits_x = reciprocal_best_hits(similarities_x)

    #### Reciprocal best hits loss ####
    inter_group_loss = InterGroupLoss(group_sizes=group_sizes, **extra_init_kwargs_loss)
    loss = inter_group_loss(reciprocal_best_hits_x, reciprocal_best_hits_y)

    assert loss.shape == ensemble_shape
    assert loss.requires_grad

    # In the following scenario, the loss should be close to -1
    extra_init_kwargs_rbh = deepcopy(extra_init_kwargs_rbh)
    extra_init_kwargs_rbh["ensemble_shape"] = None
    extra_init_kwargs_rbh["tau"] = 1e-4
    reciprocal_best_hits = ReciprocalBestHits(group_sizes=group_sizes, **extra_init_kwargs_rbh)
    reciprocal_best_hits_x = reciprocal_best_hits(similarities_y)
    loss = inter_group_loss(reciprocal_best_hits_x, reciprocal_best_hits_y)

    torch.testing.assert_close(loss, torch.tensor(-1.))

    #### Mirrortree-like loss ####
    intra_group_loss = IntraGroupLoss(**extra_init_kwargs_loss)
    loss = intra_group_loss(similarities_x, similarities_y)

    assert loss.shape == ensemble_shape
    assert loss.requires_grad

    # In the following scenario, the loss should be close to -1
    loss = intra_group_loss(similarities_y, similarities_y)

    torch.testing.assert_close(loss, torch.tensor(-1.))


test_sequence_similarity_losses( 
    length_x=3,
    length_y=4,
    alphabet_size=3,
    group_sizes=[3, 2, 4],
    extra_init_kwargs_rbh={
        "ensemble_shape": (3, 3),
        "tau": torch.tensor([1., 0.1, 3.]),
        "tau_dim_in_ensemble": 1,
    },
    extra_init_kwargs_loss={
        "score_fn": None
    }
)